<h1><center>Instagram Scraper</center></h1>

### Imports

In [15]:
###                               ETL



#Extraction libreries
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import time

#Transpose-Libraries
import pandas as pd

#Load-Libraries
import sqlite3
import datetime
from openpyxl import load_workbook
import random

In [16]:
today = datetime.date.today()
yesterday  = today-datetime.timedelta(days = 1)
the_day_before_yesterday = today-datetime.timedelta(days = 2)
timeSleep = 2
InstaPeople = ['billieeilish','justinbieber','katyperry','realdonaldtrump','b.netanyahu','melaniekroll','leelevi','margauxdietz','hollyjboo','neleburgemstr']

### General Functions

In [17]:
def day(last_sheet_name):
    
    if last_sheet_name == str(yesterday):
        return str(today)
    
    # If didn't parse yesterday -save on yesterday's date.
    elif last_sheet_name == str(the_day_before_yesterday):
        return str(yesterday)
    
    else:  # Save as 'lags'.
        return 'This sheet lags,scraped in '+str(today)

In [18]:
def commentFinder(numComment=0):
    
    #numComment = 0 takes the head comment.
    
    return '/html/body/div[6]/div[2]/div/article/div[3]/div[1]/ul/ul[{}]/div/li/div/div[1]/div[2]'.format(numComment)

In [19]:
#general photo scraping
def photoFinder(row,col):
    
    if col not in [1,2,3]:
        col = 3
        print('photo colum has been changed to 3 ')
    
    try:
        Xpath =  '//*[@id"react-root"]/section/main/div/div[4]/article/div[1]/div/div[{0}]/div[{1}]/a/div/div[2]'.format(row,col)
        buttonGrab(Xpath).click()
  
    except:
        Xpath =  '//*[@id="react-root"]/section/main/div/div[3]/article/div[1]/div/div[{0}]/div[{1}]/a/div/div[2]'.format(row,col)
        buttonGrab(Xpath).click()

In [20]:
# Button-Grab-and-Wait -Function
def buttonGrab(Xpath):
    return WebDriverWait(driver,10).until(expected_conditions.element_to_be_clickable((By.XPATH, Xpath)))

### Extract Password from Local SQL-DataBase

In [21]:
connection = sqlite3.connect("Scrpaer.db") 
crsr = connection.cursor() 

crsr.execute("SELECT * FROM user where site like 'facebook'") 
fetched = crsr.fetchall()

connection.commit() 
connection.close() 

### In-Use Function

In [22]:
searchBoxXpath = '//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input'
resultXpath = '//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/div[4]/div/a[1]/div'

In [23]:
def searcher(numResult=1):
    
    searchBox = buttonGrab(searchBoxXpath)
    searchBox.clear()
    searchBox.send_keys(person)
       
    buttonGrab(resultXpath).click()
    time.sleep(timeSleep)

In [24]:

def scrapData():
    
    
    """
    scarps data
    """
    
    nameXpath      = '//*[@id="react-root"]/section/main/div/header/section/div[1]/h2'
    followersXpath = '//*[@id="react-root"]/section/main/div/header/section/ul/li[2]/a/span'
    followingXpath = '//*[@id="react-root"]/section/main/div/header/section/ul/li[3]/a/span'
    numPostsXpath  = '//*[@id="react-root"]/section/main/div/header/section/ul/li[1]/span/span'
    
    nameOnInstagram = buttonGrab(nameXpath).text
    followers = buttonGrab(followersXpath).text
    following = buttonGrab(followingXpath).text
    numPosts  = buttonGrab(numPostsXpath).text
    
    print([person,following,followers,numPosts])
    
    return [person,following,followers,numPosts,nameOnInstagram] 
    

In [25]:
def validate(person,nameOnInstagram):
        
    if person != nameOnInstagram:
        
        nameValidation.append([person,nameOnInstagram])

In [26]:
def scrap(person):
    
    try:   
        return scrapData()
    
    except:
        # Add unsucssefull person to a list.
        if person not in unsuccessParsing:
            unsuccessParsing.append(person)
        return ['','']
    
    # For the manual process- re() func.
    finally:
        global x 
        x = person

### Start

In [27]:
path = r'C:\Users\97250\Desktop\studied\R ,python\chromedriver.exe'
driver = webdriver.Chrome(path)
driver.get("https://www.instagram.com/")
assert "Instagram" in driver.title

In [28]:
#the Facebook button
facebookXPath ='//*[@id="loginForm"]/div/div[5]/button/span[2]'
buttonGrab(facebookXPath).click()

In [29]:
username = buttonGrab('//*[@id="email"]')
password = buttonGrab('//*[@id="pass"]')
login =    buttonGrab('//*[@id="loginbutton"]')

In [30]:
username.clear()
password.clear()

username.send_keys(fetched[0][0])
password.send_keys(fetched[0][1])

login.click()

In [31]:
#  Skip buttons

try:
    continueAsXpath = '//*[@id="u_0_z"]/div[1]/div/div/div[3]/button[1]'
    buttonGrab(continueAsXpath).click()
    
except:
    pass

#click not now ver 2
try:
    notNowXPath = '/html/body/div[4]/div/div/div/div[3]/button[2]'
    buttonGrab(notNowXPath).click()

except:
    pass

In [32]:
data = []
nameValidation = []
unsuccessParsing = []
row=['','']   # Meant to deal 1st Iteration

## DO THE MAGIC

In [33]:
for person in InstaPeople:

    lastScraped = row.copy()
    
    searcher(person)
    
    row = scrap(person)
    

    # Preventing matches
    loop = 0        
    while  row[1:] == lastScraped[1:]:

        time.sleep(5)
        
        #Additional steps for insuring clear field
        searchBox = buttonGrab(searchBoxXpath)
        searchBox.clear()
        
        searcher(person)
        row = scrap(person)

        loop+=1
        if loop>3:
            break

    if len(row) >2:
        data.append(row[:-1])

    validate(row[0],row[-1])

['justinbieber', '515', '154m', '5,938']
['katyperry', '637', '109m', '1,701']
['realdonaldtrump', '8', '24.5m', '6,061']
['b.netanyahu', '10', '900k', '4,119']
['melaniekroll', '189', '221k', '616']
['leelevi', '2,509', '146k', '2,609']
['margauxdietz', '2,725', '401k', '3,180']
['hollyjboo', '1,010', '51.5k', '1,142']
['neleburgemstr', '2,737', '36.6k', '452']


In [34]:
driver.close()

In [35]:
print()
print()

print('Name was changed :' ,nameValidation)
print('Parse Error :', unsuccessParsing)

print()



Name was changed : []
Parse Error : ['billieeilish', 'justinbieber']



## Transpose

In [50]:
dataWorkFlow = data.copy() 
dataWorkFlow = pd.DataFrame(dataWorkFlow,columns = ['person','following','followers','numPosts'])
dataWorkFlow = dataWorkFlow.set_index('person')

In [51]:
def convertor(x):
    
    def remover(x):
        x = x.replace('k','').replace('m','').replace(',','')
        return float(x)
    
    x = str(x)
    
    if 'k' in x:
        return int(remover(x)*1000)
        
    if 'm' in x:
        return int(remover(x)*1000000)
    
    else:
        return int(remover(x))

dataWorkFlow['followers'] = dataWorkFlow['followers'].apply(convertor)
dataWorkFlow['following'] = dataWorkFlow['following'].apply(convertor)
dataWorkFlow['numPosts']  = dataWorkFlow['numPosts'].apply(convertor)


In [52]:
dataWorkFlow

,following,followers,numPosts
person,,,
justinbieber,515,154000000,5938
katyperry,637,109000000,1701
realdonaldtrump,8,24500000,6061
b.netanyahu,10,900000,4119
melaniekroll,189,221000,616
leelevi,2509,146000,2609
margauxdietz,2725,401000,3180
hollyjboo,1010,51500,1142
neleburgemstr,2737,36600,452


##  Save to XL

In [39]:
# Saving datas to ExcelSheet

path = r'C:\Users\97250\Desktop\studied\R ,python\InstaScraping2.xlsx'       
book = load_workbook(path)

### checks if no sheet from today
last_sheet_name = book.get_sheet_names()[-1]

if last_sheet_name == str(today) :
    last_sheet = book.get_sheet_by_name(last_sheet_name)
    book.remove_sheet(last_sheet)

writer = pd.ExcelWriter(path, engine = 'openpyxl')
writer.book = book    

#the sheet name will be the str(current date)
dataWorkFlow.to_excel(writer ,sheet_name = day(last_sheet_name)) 

writer.save()
writer.close()

<ipython-input-39-77948bcc5f45>:7: DeprecationWarning: Call to deprecated function get_sheet_names (Use wb.sheetnames).
  last_sheet_name = book.get_sheet_names()[-1]
<ipython-input-39-77948bcc5f45>:10: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  last_sheet = book.get_sheet_by_name(last_sheet_name)
<ipython-input-39-77948bcc5f45>:11: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  book.remove_sheet(last_sheet)
C:\Users\97250\anaconda3\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


###  Save to SQL 

In [55]:
dataWorkFlow['date'] = str(today)

dataWorkFlow = dataWorkFlow.reset_index().values.tolist()

In [58]:
# connecting to the database  
connection = sqlite3.connect("Scrpaer.db") 
# cursor  
crsr = connection.cursor() 

sql = "INSERT INTO Instagram VALUES (?, ?, ?, ?, ?)"
connection.executemany(sql, dataWorkFlow) 

###  Fetch Data
crsr.execute("SELECT * FROM instagram")  

# store all the fetched data in 'ans' variable 
fetched = crsr.fetchall()   
pd.DataFrame(fetched,columns=['person','following','followers','numPosts','date']).set_index('person')

connection.commit() 
connection.close() 

In [62]:
fetched[-9:]

[('justinbieber', '515', '154000000', '5938', '2020-12-25'),
 ('katyperry', '637', '109000000', '1701', '2020-12-25'),
 ('realdonaldtrump', '8', '24500000', '6061', '2020-12-25'),
 ('b.netanyahu', '10', '900000', '4119', '2020-12-25'),
 ('melaniekroll', '189', '221000', '616', '2020-12-25'),
 ('leelevi', '2509', '146000', '2609', '2020-12-25'),
 ('margauxdietz', '2725', '401000', '3180', '2020-12-25'),
 ('hollyjboo', '1010', '51500', '1142', '2020-12-25'),
 ('neleburgemstr', '2737', '36600', '452', '2020-12-25')]